Visual Detection of Surfers in a Scene Using a Trained Surfer Detection Model
---------

This notebook will walk you through the steps necessary to generate an annotated scene with green bounding boxes that denote the presence of surfers, as determined by surferdetection_predict.py.

This routine will build a "detection map" and overlay it on an input scene.  Note that it creates a temporary directory of tiles from the input image to pass to surferdetection_predict.py and deletes recursively.

The path to a restored Tensorflow model is contained in surferdetection_predict.py.


In [ ]:
from __future__ import print_function
from __future__ import division

import os
import re
import shutil
import time

from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import surferdetection_predict

# LOCATION OF SCENES
scene_dir = "surferdetection_predict"
tmp_dir = os.path.join(scene_dir,"tmp") #temp directory for tiling op

# SCENE FILENAME
scene_filename = "testscene_0.jpeg"

# SCENE INFORMATION
scene_height = 720
scene_width = 1280
pixel_depth = 255  # Number of levels per pixel.
num_channels = 3 #channels for color
tile_size = 80  # Pixel width and height.

# DESIRED DETECTION MAP SETTINGS
border_size = 3 # size of bounding box
color = 255 # intensity of green channel color
confidence_threshold = 0.60 # only detect if softmax (probability) is greater than this percentage
probability = True # set to True to use threshold
ignore_tiles = np.append(range(64),range(112,144)) # tiles to ignore in the output detection map

Tiling Routine
-------------

Creates numpy array from input image.

Creates temporary directory to hold the created tiles.

Passes temporary directory to surferdetection_predict.py.

Draws bounding boxes on the numpy array representing positive detecetions in the scene.

Plots result inline as PIL Image.

Saves result to lossless PNG in the scene directory.

Cleans up temporary directory.

In [ ]:
input_scene = os.path.join(scene_dir,scene_filename)

# Bring in the scene
img = Image.open(input_scene)
img = np.array(img)

print ("Prediction routine started... standby.")


# Make a temp directory for tiles
if os.path.isdir(tmp_dir): shutil.rmtree(tmp_dir)
os.makedirs(tmp_dir)

# Tile only the region of interest and save to the temporary directory
tiles_per_row = int(scene_width/tile_size) #16 for tiles of size 80
tiles_per_column = int(scene_height/tile_size) #9 for tiles of size 80
tiles_per_scene = tiles_per_row * tiles_per_column #144 for tiles of size 80

tile_index = 0  
for i in range(tiles_per_column): #for the row   
  for j in range(tiles_per_row): #for the column
    if tile_index in ignore_tiles:
      tile_index += 1  
      continue
    tile = img[i*tile_size:(i+1)*tile_size,j*tile_size:(j+1)*tile_size,:]
    tile = Image.fromarray(tile)
    tile.save(os.path.join(tmp_dir,'tile_%d.jpeg' % tile_index))
    tile_index += 1

    
# Initialize numpy array to hold predictions
prediction_map = np.zeros(tiles_per_scene,dtype=int)

# predict tiles from surferdetection_predict.predict()
start = time.time()
use_tiles = np.setdiff1d(np.array(range(tiles_per_scene)),ignore_tiles) # un-ignored tiles that will be used for output
prediction_map[use_tiles] = surferdetection_predict.predict(tmp_dir, 
                                                            threshold = confidence_threshold,
                                                            prob = probability)
end = time.time()


# get the indices in the image where to draw the boxes
reshaped_predictions = np.reshape(prediction_map,(tiles_per_column,tiles_per_row)).astype(int)
box_index=[]

for i in xrange(0,len(np.nonzero(reshaped_predictions)[0])):
  index = (tile_size * np.nonzero(reshaped_predictions)[0][i],
           tile_size * np.nonzero(reshaped_predictions)[1][i])
  box_index.append(index)

  # "draw" the boxes by replacing pixels with bounding boxes
  for i in xrange(0,len(box_index)):
    img[box_index[i][0]: box_index[i][0] + border_size,  #top border
        box_index[i][1]: box_index[i][1] + tile_size,
        1] = color
    img[box_index[i][0]: box_index[i][0] + tile_size,  #left border
        box_index[i][1]: box_index[i][1] + border_size,
        1] = color
    img[box_index[i][0] + (tile_size - border_size): box_index[i][0] + tile_size,  #bottom border
        box_index[i][1]: box_index[i][1] + tile_size,
        1] = color
    img[box_index[i][0]: box_index[i][0] + tile_size,  #right border
        box_index[i][1] + (tile_size - border_size): box_index[i][1] + tile_size,
        1] = color
    
# save the annotated image out
img = Image.fromarray(img)
img.save(os.path.join(scene_dir,("output_image.png")))

# Delete temp tiles recursively
shutil.rmtree(tmp_dir)
    
# print some status statistics
print ("Prediction time for the scene: %.2f seconds" %(end - start))
print("Detection map completed.")

# display the image
plt.imshow(img)
plt.show()